In [1]:
import glob
import random
import math
import h5py

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from datetime import datetime
import time
import os
import sys
from pprint import pprint

from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [2]:
data_path = "D:/Coding/Thesis/Data/STFT Output/**/*.h5"
data_files = glob.glob(data_path, recursive=True)

# General data config

In [3]:
config = {}

config['model_folder'] = 'D:/Coding/Thesis/movement_prediction/Models/'

config['EEG_window_length_in_ms'] = 2000

config['delta_time_k'] = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
config['tap_count_times_p'] = np.array([0.5, 1, 5, 10, 50, 100, 500])

config['n_output_features'] = np.sum([config['delta_time_k'].shape, config['tap_count_times_p'].shape])

config['read_config_from_h5'] = True

config['EEG_sampling_rate'] = 1000
config['stft_stride'] = 128
config['sampling_rate_after_stft'] = config['EEG_sampling_rate'] / config['stft_stride']
config['sample_length_after_stft'] = 1000 / config['sampling_rate_after_stft']

config['tap_count_times_in_samples'] = np.multiply(config['tap_count_times_p'], config['EEG_sampling_rate'])

# Load entire participant's data into memory

In [4]:
def load_data(h5_files):
    EEG, taps = [], []
    
    # Makes sure that if a single file is passed in, it is put into a list
    if type(h5_files) != 'list':
        h5_files = [h5_files]
    
    for f in h5_files:
        with h5py.File(f, 'r') as f_open:
            if config['read_config_from_h5']:
                    config['EEG_sampling_rate'] = f_open.attrs['original_sampling_rate']
                    config['stft_stride'] = f_open.attrs['stft_hopsize']
            
            for session in list(f_open.keys()):
                for activity_window in list(f_open[session].keys()):
                    if activity_window.startswith('window_'):
                        EEG.append(np.array(f_open[session][activity_window]['stft'], dtype='float32'))
                        taps.append(np.array(f_open[session][activity_window]['taps']))
    
    ############# Inf found in data!
    for i in EEG:
        i[np.isinf(i)] = 0
    
    return (EEG, taps)

stft, taps_data = load_data('D:\\Coding\\Thesis\\Data\\STFT Output\\DS99.h5')


# Add dynamic config parameter
config['window_length_sftf_samples'] = np.ceil(config['EEG_window_length_in_ms'] / 1000 * config['EEG_sampling_rate'] / config['stft_stride']).round().astype(np.int16)[0]

print(config['window_length_sftf_samples'])
print(f'Number of activity windows: {len(stft)}')
print([x.shape for x in stft])
print([y.shape for y in taps_data])

16
Number of activity windows: 7
[(64, 12300, 11), (64, 17318, 11), (64, 941, 11), (64, 802, 11), (64, 649, 11), (64, 649, 11), (64, 7450, 11)]
[(1, 698), (1, 876), (1, 50), (1, 54), (1, 4), (1, 3), (1, 516)]


# Normalize input

In [5]:
def normalize_input(_input, normalize_over_eeg_channels=False):
    '''stft: A list of numpy arrays.
    
   Normalize over all features, ignoring  
    
    '''
    stft_cat = np.concatenate(_input, axis=1)
    
    print(f'NaN: {np.sum(np.isnan(stft_cat))}')
    print(f'Inf: {np.sum(np.isinf(stft_cat))}')
    
    if normalize_over_eeg_channels:
        axes = (0, 1)
    else:
        axes = 1
    
    means = np.mean(stft_cat, axis=axes, keepdims=True)
    st_devs = np.std(stft_cat, axis=axes, keepdims=True)
    
    out = [(i - means) - st_devs for i in _input]
    
    return out
    

stft_norm = normalize_input(stft)

#print([x.shape[1] for x in stft_norm])

# Remove end of EEG activity, since there is no activity of interest

seconds_to_remove = 28
samples_to_remove = np.ceil(config['sampling_rate_after_stft'] * seconds_to_remove).astype(np.int)

stft_norm_data = [x[:, :-samples_to_remove, :] for x in stft_norm]

NaN: 0
Inf: 0


# Split data into train, validation, and test set

For every window where taps were recorded, we use the first 80% of activity as test set. We then split the remaining 20% in half to assign to validation and test set, respectively.

In [6]:
stft_norm = {}
stft_norm['train'] = []
stft_norm['validation'] = []
stft_norm['test'] = []

taps = {'train': [], 'validation': [], 'test': []}

for i, x in enumerate(stft_norm_data):
    x_len = x.shape[1]
    
    # Check if activity window length is too short.
    # If so, add it to train set.
    # If extremely short, discard window
    if taps_data[i].shape[1] < 5:
        continue
    
    train_idx = np.arange(0, (x_len//10 * 8))

    
    validation_idx = np.arange((x_len//10 * 8), (x_len//10 * 9))
    test_idx = np.arange((x_len//10 * 9), x_len)
    
    stft_norm['train'].append(x[:, train_idx, :])
    stft_norm['validation'].append(x[:, validation_idx, :])
    stft_norm['test'].append(x[:, test_idx, :])
    
    # Add taps
    _taps = taps_data[i] / config['stft_stride']

    _taps_train = _taps[np.logical_and(_taps > (train_idx[0]), \
                                       _taps < (train_idx[-1]))]
    
    _taps_vali = _taps[np.logical_and(_taps > (validation_idx[0]), \
                                      _taps < (validation_idx[-1]))]
    
    _taps_test = _taps[np.logical_and(_taps > (test_idx[0]), \
                                      _taps < (test_idx[-1]))]
    
    # Adjust tap timings to zero at beginning of sequence
    _taps_train = _taps_train - (train_idx[0])
    _taps_vali = _taps_vali - (validation_idx[0])
    _taps_test = _taps_test - (test_idx[0])
    
    taps['train'].append(np.array(_taps_train, dtype=np.int))
    taps['validation'].append(np.array(_taps_vali, dtype=np.int))
    taps['test'].append(np.array(_taps_test, dtype=np.int))
    
print(f'Total taps:\t {[y.shape[1] for y in taps_data]}')
print(f'Train set:\t {[y.shape for y in taps["train"]]}')
print(f'Valid set:\t {[y.shape for y in taps["validation"]]}')
print(f'Test set:\t {[y.shape for y in taps["test"]]}')

Total taps:	 [698, 876, 50, 54, 4, 3, 516]
Train set:	 [(545,), (656,), (48,), (35,), (373,)]
Valid set:	 [(63,), (111,), (0,), (5,), (68,)]
Test set:	 [(90,), (109,), (2,), (14,), (75,)]


# Generating a trainable data set

Now, that we have split our data set into train, validation, and test set, we need to transform the timeseries data into a matrix of size $M \times N$ with $M = \text{number of samples}$ and $N = |\text{features}| \times \text{window length}$.

Each row is the input data shifted over by one sample. Columns is the entire input data for the window, but flattened into a vector.

In [7]:
for _set in stft_norm.items():
    current_set = _set[1]

    for i, row in enumerate(current_set):
        current_set[i] = np.transpose(row, (1, 0, 2))
        current_set[i] = np.reshape(current_set[i], (current_set[i].shape[0], -1))

## Tap data generation functions

In [8]:
def get_taps_in_window(taps, window_end):      
    tap_deltas = get_delta_taps(taps, window_end)

    future_tap_n = get_n_future_taps(taps, window_end)

    result = np.concatenate((tap_deltas, future_tap_n))

    # Convert values to log to compress values
    result = [np.log10(tap) if tap > 0 else tap for tap in result]

    threshold_ms = 1
    result = np.maximum(result, np.log10(threshold_ms))

    return result
    
    
def get_delta_taps(taps, window_idx):
    n_k = len(config['delta_time_k'])

    next_kth_taps = taps[taps > window_idx][:n_k]

    # Ensure that if not enough taps were found the array is padded with 0s.
    # This only occurs at the end of a window
    if len(next_kth_taps) < n_k:
        next_kth_taps = np.concatenate((next_kth_taps, np.zeros(n_k - len(next_kth_taps))))

    tap_deltas = next_kth_taps - window_idx

    return tap_deltas * config['stft_stride']


def get_n_future_taps(taps, window_idx):
    n_future_taps = np.zeros(len(config['tap_count_times_in_samples']))

    for p_idx, p in enumerate(config['tap_count_times_in_samples']):
        n_future_taps[p_idx] = len(
            taps[
                (taps > window_idx) &
                (taps <= (window_idx + p))
            ]
        )

    return n_future_taps

def generate_trainable_data(X: dict, Y: dict, key: str, config: dict, stride=1, verbose=False):
    '''
    Returns the trainable data in matrix form.
    
    X and Y are dictionaries holding the raw training, validation, and test data.
    key: A string specifying the key to access in X and Y
    stride: If stride is n > 1, every n-th sample is taken. Higher stride values will result in less memory usage, but also less accurate training/predictions.
    '''
    win_length = config['window_length_sftf_samples']

    window_lengths = [x.shape[0] // stride for x in X[key]]

    data_set_X = None # Clear memory before reassignment
    data_set_X = np.zeros([np.sum(window_lengths), X[key][0].shape[1] * win_length], dtype=np.float32)
    data_set_Y = np.zeros([data_set_X.shape[0], config['n_output_features']], dtype=np.uint)

    if verbose:
        print(f'Training X array has shape: {data_set_X.shape}')
        print(f'X data array uses {data_set_X.nbytes // 1024 // 1024} MiB, {data_set_X.itemsize} bytes per item')
        print(f'Training Y array has shape: {data_set_Y.shape}')
        print(f'Y data array uses {data_set_Y.nbytes // 1024} KiB, {data_set_Y.itemsize} bytes per item')
    
    i = 0

    for win_idx, stft_win in enumerate(X[key]):
        for sample_idx in range(win_length, stft_win.shape[0], stride):
            data_set_X[i, :] = stft_win[(sample_idx - win_length):sample_idx, :].flatten()
            data_set_Y[i, :] = get_taps_in_window(taps[key][win_idx], sample_idx)

            i += 1
            
    return data_set_X, data_set_Y

In [9]:
train_stride = 1

train_set_X, train_set_Y = None, None
train_set_X, train_set_Y = generate_trainable_data(stft_norm, taps, 'train', config, stride=train_stride, verbose=True)

# Converting to Fortran-contiguous array improves speed when fitting model
train_set_X = np.asfortranarray(train_set_X, dtype=np.float32)

Training X array has shape: (30160, 11264)
X data array uses 1295 MiB, 4 bytes per item
Training Y array has shape: (30160, 17)
Y data array uses 2002 KiB, 4 bytes per item


# Linear Model

In [ ]:
# For now, let's save models in a list
lm = []

! Should replace linear_model.Lasso with linear_mode.LassoCV

In [ ]:
#for feature in range(train_set_Y.shape[1]):
for feature in range(1): # Only use range(1) for debugging    
    start = time.time()
    print(f'Training feature {feature} ... ')
    
    # Fit model
    lm.append(linear_model.Lasso(n_alphas=1, copy_X=False, cv=1, n_jobs=3, verbose=True))
    lm[-1].fit(train_set_X, train_set_Y[:, feature])
    
    time_elapsed = time.time() - start
    print(f'Done in {time_elapsed:.2f} seconds.')
    
train_set_X, train_set_Y = None, None

In [ ]:
test_stride = 1

test_set_X, test_set_Y = None, None
test_set_X, test_set_Y = generate_trainable_data(stft_norm, taps, 'test', config, stride=test_stride, verbose=False)
prediction = lm[0].predict(test_set_X)

In [17]:
RMSE = mean_squared_error(test_set_Y[:, 0], prediction, squared=False)
print(f'RMSE: {RMSE:.3f}')

RMSE: 0.886


In [ ]:
# New plots for new test framework
%matplotlib inline

#RMSE = np.sqrt(((_Y_hat[0] - _Y[0]) ** 2).mean(0))


#fig, ((deltas_plt, n_taps_plt), (RMSE_deltas_plt, RMSE_ntaps_plt)) = plt.subplots(2, 2, figsize=(15, 15))

plt.figure()
fig, axs = plt.subplots(len(_Y), 1, figsize=(15, 5 * len(_Y)))


for sequence_id in range(len(_Y)):
    axs[sequence_id].plot(_Y_hat[sequence_id][:, output_feature], '--')
    axs[sequence_id].plot(_Y[sequence_id][:, output_feature], '-')
    #axs[sequence_id].plot(_Y_random[:, output_feature], '-')
    axs[sequence_id].set_xlabel('Time in samples')
    axs[sequence_id].set_ylabel('$\Delta t$ in ms')

plt.show()